In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import copy

import tensorflow as tf

from google.protobuf import text_format
from tensorflow.python.client import graph_util

In [9]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string("retrained_graph.pb", "",
                           """TensorFlow 'GraphDef' file to load.""")
tf.app.flags.DEFINE_boolean("input_binary", False,
                            """Whether the input files are in binary format.""")
tf.app.flags.DEFINE_string("output_graph", "",
                           """Output 'GraphDef' file name.""")
tf.app.flags.DEFINE_string("input_node_names", "",
                           """The name of the output nodes, comma separated.""")
tf.app.flags.DEFINE_string("output_node_names", "",
                           """The name of the output nodes, comma separated.""")
tf.app.flags.DEFINE_integer("placeholder_type_enum",
                            tf.float32.as_datatype_enum,
                            """The AttrValue enum to use for placeholders.""")


def strip_unused(input_graph, input_binary, output_graph, input_node_names,
                 output_node_names, placeholder_type_enum):
  """Removes unused nodes from a graph."""

  if not tf.gfile.Exists(input_graph):
    print("Input graph file '" + input_graph + "' does not exist!")
    return -1

  if not output_node_names:
    print("You need to supply the name of a node to --output_node_names.")
    return -1

  input_graph_def = tf.GraphDef()
  mode = "rb" if input_binary else "r"
  with tf.gfile.FastGFile(input_graph, mode) as f:
    if input_binary:
      input_graph_def.ParseFromString(f.read())
    else:
      text_format.Merge(f.read(), input_graph_def)

  # Here we replace the nodes we're going to override as inputs with
  # placeholders so that any unused nodes that are inputs to them are
  # automatically stripped out by extract_sub_graph().
  input_node_names_list = input_node_names.split(",")
  inputs_replaced_graph_def = tf.GraphDef()
  for node in input_graph_def.node:
    if node.name in input_node_names_list:
      placeholder_node = tf.NodeDef()
      placeholder_node.op = "Placeholder"
      placeholder_node.name = node.name
      placeholder_node.attr["dtype"].CopyFrom(tf.AttrValue(
          type=placeholder_type_enum))
      inputs_replaced_graph_def.node.extend([placeholder_node])
    else:
      inputs_replaced_graph_def.node.extend([copy.deepcopy(node)])

  output_graph_def = graph_util.extract_sub_graph(inputs_replaced_graph_def,
                                                  output_node_names.split(","))

  with tf.gfile.GFile(output_graph, "wb") as f:
    f.write(output_graph_def.SerializeToString())
  print("%d ops in the final graph." % len(output_graph_def.node))


def main(unused_args):
  strip_unused(FLAGS.input_graph, FLAGS.input_binary, FLAGS.output_graph,
               FLAGS.input_node_names, FLAGS.output_node_names,
               FLAGS.placeholder_type_enum)

DuplicateFlagError: The flag 'retrained_graph.pb' is defined twice. First from C:\Users\user\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py, Second from C:\Users\user\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py.  Description from first occurrence: TensorFlow 'GraphDef' file to load.

In [10]:
if __name__ == "__main__":
  tf.app.run()

Input graph file '' does not exist!


SystemExit: 

In [11]:
%tb

SystemExit: 